***
# TEST
***

In [ ]:
import os
import json
import pandas as pd
import pvdeg
from pytest import approx
from pvdeg import TEST_DATA_DIR

# Load weather data
WEATHER = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "weather_day_pytest.csv"), index_col=0, parse_dates=True
)
with open(os.path.join(TEST_DATA_DIR, "meta.json"), "r") as file:
    META = json.load(file)

# Load expected results
rh_expected = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "input_day_pytest.csv"), index_col=0, parse_dates=True
)
rh_cols = [col for col in rh_expected.columns if "RH" in col]
rh_expected = rh_expected[rh_cols]


def test_module():
    """
    test pvdeg.humidity.calc_rel_humidity

    Requires:
    ---------
    weather dataframe and meta dictionary
    """
    result = pvdeg.humidity.module(WEATHER, META)
    pd.testing.assert_frame_equal(result, rh_expected, check_dtype=False)


def test_psat():
    """
    test pvdeg.humidity.psat

    Requires:
    ---------
    weahter dataframe and meta dictionary
    """
    psat_avg = pvdeg.humidity.psat(temp=WEATHER["temp_air"])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)

In [ ]:
test_psat()

In [ ]:
import pvdeg
import pickle
import pandas as pd
import numpy as np
import xarray as xr
import os
from pvdeg import TEST_DATA_DIR

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)
with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), "rb") as f:
    GEO_WEATHER = pickle.load(f)

autotemplate

In [ ]:
autotemplate_result = pvdeg.geospatial.auto_template(
    func=pvdeg.humidity.module, ds_gids=GEO_WEATHER
).compute()

In [ ]:
humidity_template = xr.open_dataset(
    os.path.join(TEST_DATA_DIR, "humidity_template.nc")
).compute()

In [ ]:
def compare_datasets(ds1: xr.Dataset, ds2: xr.Dataset, atol=1e-10) -> bool:
    """Compare loaded datasets with "empty-like" values"""

    if ds1.dims != ds2.dims:
        return False

    if set(ds1.coords.keys()) != set(ds2.coords.keys()):
        return False

    for coord in ds1.coords:
        if ds1.coords[coord].dtype.kind in {"i", "f"}:
            # Use np.allclose for numeric coordinates
            if not np.allclose(ds1.coords[coord], ds2.coords[coord], atol=atol):
                return False
        elif ds1.coords[coord].dtype.kind == "M":  # datetime64 type
            # Use array equality for datetime coordinates
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False
        else:
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False

    if set(ds1.data_vars.keys()) != set(ds2.data_vars.keys()):
        return False

    for var in ds1.data_vars:
        if not np.allclose(ds1[var], ds2[var], atol=atol):
            return False

    for dim in ds1.dims:
        if not ds1.indexes[dim].equals(ds2.indexes[dim]):
            return False

    return True

In [ ]:
assert pvdeg.utilities.compare_datasets(autotemplate_result, humidity_template)

output template

In [ ]:
shapes = {
    "RH_surface_outside": ("gid", "time"),
    "RH_front_encap": ("gid", "time"),
    "RH_back_encap": ("gid", "time"),
    "RH_backsheet": ("gid", "time"),
}

manual_template = pvdeg.geospatial.output_template(
    shapes=shapes, ds_gids=GEO_WEATHER
).compute()

In [ ]:
pvdeg.utilities.compare_datasets(manual_template, humidity_template)

In [ ]:
# test template

shapes = {"testA": ("gid",), "testB": ("gid", "time")}

template = pvdeg.geospatial.output_template(
    shapes=shapes,
    ds_gids=GEO_WEATHER,
)

In [ ]:
template.to_netcdf(os.path.join(TEST_DATA_DIR, "mismatch-template.nc"))

In [ ]:
import pvdeg
from pvdeg import TEST_DATA_DIR
import pickle
import pandas as pd
import numpy as np
import xarray as xr
import os

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)

with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), "rb") as f:
    GEO_WEATHER = pickle.load(f).compute().load()

HUMIDITY_TEMPLATE = xr.open_dataset(
    os.path.join(TEST_DATA_DIR, "humidity_template.nc"), engine="h5netcdf"
).compute()

In [ ]:
GEO_WEATHER

In [ ]:
GEO_WEATHER.chunks

In [ ]:
HUMIDITY_TEMPLATE.chunks

In [ ]:
shapes = {
    "RH_surface_outside": ("gid", "time"),
    "RH_front_encap": ("gid", "time"),
    "RH_back_encap": ("gid", "time"),
    "RH_backsheet": ("gid", "time"),
}

# falsely assigning chunks here
manual_template = pvdeg.geospatial.output_template(shapes=shapes, ds_gids=GEO_WEATHER)

assert pvdeg.utilities.compare_templates(manual_template, HUMIDITY_TEMPLATE)
for k, v in manual_template.chunks.items():
    if len(v) != 1:
        raise ValueError(f"""
                          Need one chunk per axis for an unchunked input
                          dimension {k} has {len(v)} chunks.
                          """)

In [ ]:
chunked_weather = GEO_WEATHER.chunk({"gid": 3})

In [ ]:
HUMIDITY_TEMPLATE.chunk({"gid": 3})

In [ ]:
chunked_template

In [ ]:
pvdeg.utilities.compare_templates(chunked_template, HUMIDITY_TEMPLATE.chunk({"gid": 3}))

In [ ]:
chunked_template = pvdeg.geospatial.auto_template(
    ds_gids=chunked_weather, func=pvdeg.humidity.module
)

geo_res = pvdeg.geospatial.analysis(
    weather_ds=chunked_weather,
    meta_df=GEO_META,
    func=pvdeg.humidity.module,
    template=chunked_template,
)

In [ ]:
geo_res = pvdeg.geospatial.analysis(
    chunked_weather,
    meta_df=GEO_META,
    func=pvdeg.humidity.module,
)

In [ ]:
res_ds = pvdeg.geospatial.analysis(
    weather_ds=GEO_WEATHER,
    meta_df=GEO_META,
    func=pvdeg.standards.standoff,
)

data_var = res_ds["x"]

# Stack the latitude and longitude coordinates into a single dimension
# convert to dataframe, this can be done with xr.dataset.to_dataframe as well
stacked = data_var.stack(z=("latitude", "longitude"))
latitudes = stacked["latitude"].values
longitudes = stacked["longitude"].values
data_values = stacked.values
combined_array = np.column_stack((latitudes, longitudes, data_values))

res = pd.DataFrame(combined_array).dropna()
ans = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-standoff-res.csv"), index_col=0)
res.columns = ans.columns

# pd.testing.assert_frame_equal(res, ans, check_dtype=False, check_names=False)

In [ ]:
res_ds.chunks

### Diffusion TESTing

In [17]:
import os
import pandas as pd
import numpy as np
import pytest
import pvdeg
from pvdeg import TEST_DATA_DIR
import json

In [ ]:
WEATHER = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "weather_day_pytest.csv"), index_col=0, parse_dates=True
)
with open(os.path.join(TEST_DATA_DIR, "meta.json"), "r") as file:
    META = json.load(file)

In [ ]:
WEATHER

In [ ]:
META

In [ ]:
temperature = pvdeg.temperature.temperature(
    weather_df=WEATHER,
    meta=META,
    cell_or_mod="module", 
    temp_model="sapm",
    conf="open_rack_glass_polymer",
)

temperature = pd.DataFrame(temperature, columns = ['module_temperature'])
temperature['time'] = list(range(len(temperature)))

In [ ]:
temperature

In [ ]:
pressure = 0.2109 * (1 - 0.0065 * META['altitude'] / 288.15) ** 5.25588

oxygen_profile = pvdeg.diffusion.esdiffusion(
    temperature=temperature, 
    edge_seal='OX005', 
    encapsulant='OX003', 
    edge_seal_width=1.5, 
    encapsulant_width=10, 
    seal_nodes=20, 
    encapsulant_nodes=50, 
    press=pressure, 
    repeat=2
)

In [ ]:
oxygen_profile

In [ ]:
oxygen_profile.to_csv("1d-oxygen-profile.csv")

In [ ]:
res = pd.read_csv(os.path.join(TEST_DATA_DIR, "1d-oxygen-profile.csv"), index_col=0, dtype="float64")

In [ ]:
from copy import copy

col_list = copy(res.columns).values
col_list[21] = "1.5"

res.columns = col_list.astype(float)

res.columns

In [ ]:
pd.testing.assert_frame_equal(
    oxygen_profile, res, 
    check_dtype=False, 
    check_column_type=False, 
)

### Fixing Kempe Gap Calc broken file changes

In [1]:
from pvdeg.scenario import Scenario
from pvdeg.standards import standoff
from pvdeg import TEST_DATA_DIR
import pvdeg
import json
import pandas as pd
import pytest
import os

# problems with scenario creating directory in test directory?
EMAIL = "user@mail.com"
API_KEY = "DEMO_KEY"

In [6]:
def test_Scenario_add():

    a = Scenario(name="test")
    a.clean()
    a.restore_credentials(email=EMAIL, api_key=API_KEY)
    a.addLocation(lat_long=(40.63336, -73.99458))
    a.addModule(module_name="test-module")
    a.addJob(func=standoff, func_kwarg={"wind_factor": 0.35})

    restored = Scenario.load_json(
        file_path=os.path.join(TEST_DATA_DIR, "test-scenario.json")
    )

    a.path, restored.path = None, None
    a.file, restored.file = None, None

    assert a == restored

In [1]:
import pvdeg

In [ ]:
pvdeg.utilities.pvdeg_datafiles

In [19]:
def test_read_material_special():

    template_material = pvdeg.utilities.read_material(pvdeg_file="AApermeation", key="AA000")

    assert len(template_material) == 1
    assert "comment" in template_material

test_read_material_special()

In [ ]:
template_material = pvdeg.utilities.read_material(pvdeg_file="AApermeation", key="AA000")

[type(x) for x in template_material.values()][0]

In [21]:
def test_read_material_normal():

    res = {
        'name': 'ST504', 
        'alias': 'PET1', 
        'contributor': 'Michael Kempe', 
        'source': 'unpublished measurements', 
        'Fickian': True,
        'Ead': 47.603, 
        'Do': 0.554153, 
        'Eas': -11.5918, 
        'So': 9.554366e-07, 
        'Eap': 34.2011, 
        'Po': 2128.8937
    }

    template_material = pvdeg.utilities.read_material(pvdeg_file="O2permeation", key="OX002")

    assert template_material == res

test_read_material_normal()

In [22]:
def test_read_material_fewer_params():

    res = {
        'name': 'ST504', 
        'Fickian': True,
    }

    template_material = pvdeg.utilities.read_material(pvdeg_file="O2permeation", key="OX002", parameters=["name", "Fickian"])

    assert template_material == res

test_read_material_fewer_params()

In [24]:
def test_read_material_extra_params():

    res = {
        'namenotindict1': None,
        'namenotindict2': None,
    }

    template_material = pvdeg.utilities.read_material(pvdeg_file="O2permeation", key="OX002", parameters=["namenotindict1", "namenotindict2"])

    assert template_material == res


test_read_material_extra_params()

In [ ]:
def test_search_json():


In [5]:
# pvdeg_file should override fp if both are provided
def test_read_material_fp_override():

    res = {
        'name': 'ST504', 
        'alias': 'PET1', 
        'contributor': 'Michael Kempe', 
        'source': 'unpublished measurements', 
        'Fickian': True,
        'Ead': 47.603, 
        'Do': 0.554153, 
        'Eas': -11.5918, 
        'So': 9.554366e-07, 
        'Eap': 34.2011, 
        'Po': 2128.8937
    }

    from pvdeg import DATA_DIR

    # pass pvdeg file and it gets overridden by the file path
    template_material = pvdeg.utilities.read_material(
        pvdeg_file="O2permeation", 
        fp=os.path.join(DATA_DIR, "AApermeation.json"), 
        key="OX002",
    )

    assert template_material == res

test_read_material_fp_override()

In [9]:
def test_search_json():

    name_res = pvdeg.utilities.search_json(pvdeg_file="H2Opermeation", name_or_alias="Ethylene Vinyl Acetate")
    alias_res = pvdeg.utilities.search_json(pvdeg_file="H2Opermeation", name_or_alias="EVA")

    assert name_res == "W001"
    assert alias_res == "W001"

test_search_json()

In [3]:
from pvdeg import DATA_DIR

pvdeg_file = "H2Opermeation"
invalid_name_or_alias = "namenotindict"
expected_error_message = (
    rf"name_or_alias: {invalid_name_or_alias} not in JSON at "
    rf"{os.path.join(DATA_DIR, pvdeg.utilities.pvdeg_datafiles[pvdeg_file])}"
)

with pytest.raises(ValueError, match=expected_error_message):
    pvdeg.utilities.search_json(pvdeg_file=pvdeg_file, name_or_alias=invalid_name_or_alias)

Failed: Invalid regex pattern provided to 'match': incomplete escape \U at position 46

In [2]:

expected_error_message = (
    rf"name_or_alias: {invalid_name_or_alias} not in JSON at "
    rf"{os.path.join(DATA_DIR, pvdeg.utilities.pvdeg_datafiles[pvdeg_file])}"
)

NameError: name 'invalid_name_or_alias' is not defined

In [4]:
expected_error_message

'name_or_alias: namenotindict not in JSON at C:\\Users\\tford\\dev\\PVDegradationTools\\pvdeg\\data\\H2Opermeation.json'

In [5]:
pvdeg.utilities.search_json(pvdeg_file=pvdeg_file, name_or_alias=invalid_name_or_alias)

TypeError: 'module' object is not callable

In [9]:
import pvdeg

import xarray as xr
import pandas as pd
import numpy as np
import pickle
import os

In [28]:
def mixed_res_dict(weather_df, meta):
    """
    geospatial test function. returns have mixed dimensions. the first is a timeseries, the second is a float.
    This function is meant to discover problems with geospatial.analysis and its subroutines.

    .. code_block : Python 

        Shapes = {
            "temperatures" : ("gid", "time"),
            "avg_temp" : ("gid", ),
        }
    """

    timeseries_df = pd.DataFrame(pvdeg.temperature.module(weather_df, meta))
    avg_temp = timeseries_df[0].mean()

    return {'temperatures' : timeseries_df, 'avg_temp' : avg_temp}
    # return timeseries_df, avg_temp



In [32]:
GEO_WEATHER.isel(gid=0)

<xarray.Dataset> Size: 981kB
Dimensions:            (time: 17520)
Coordinates:
    gid                int64 8B 449211
  * time               (time) datetime64[ns] 140kB 2022-01-01 ... 2022-12-31T...
Data variables:
    temp_air           (time) float64 140kB -12.0 -12.4 -12.7 ... -3.6 -4.1 -4.1
    wind_speed         (time) float64 140kB 0.6 0.6 0.6 0.6 ... 0.8 0.7 0.6 0.6
    dhi                (time) float64 140kB 0.0 0.0 0.0 0.0 ... 106.0 47.0 22.0
    ghi                (time) float64 140kB 0.0 0.0 0.0 0.0 ... 111.0 47.0 23.0
    dni                (time) float64 140kB 0.0 0.0 0.0 0.0 ... 26.0 2.0 21.0
    relative_humidity  (time) float64 140kB 99.93 100.0 100.0 ... 100.0 100.0

array([80, 80, 80, ..., 80, 80, 80])

In [48]:
xr.Dataset(
    data_vars={
        "temperatures" : (('time'), np.full((8760,), fill_value=80)),
        "avg_temp": 80
    },
    coords={
        'time' : pd.date_range(start="2001-01-01", periods=8760, freq='1h')
    }
)

<xarray.Dataset> Size: 105kB
Dimensions:       (time: 8760)
Coordinates:
  * time          (time) datetime64[ns] 70kB 2001-01-01 ... 2001-12-31T23:00:00
Data variables:
    temperatures  (time) int32 35kB 80 80 80 80 80 80 80 ... 80 80 80 80 80 80
    avg_temp      int32 4B 80

In [36]:
mixed_res_dict(
    GEO_WEATHER.isel(gid=0).to_dataframe(),
    GEO_META.iloc[0].to_dict(),
)['temperatures'][0]

The array tilt angle was not provided, therefore the latitude tilt of 39.9 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


time
2022-01-01 00:00:00   -12.000000
2022-01-01 00:30:00   -12.400000
2022-01-01 01:00:00   -12.700000
2022-01-01 01:30:00   -13.000000
2022-01-01 02:00:00   -13.300000
                         ...    
2022-12-31 21:30:00     0.588357
2022-12-31 22:00:00     1.129164
2022-12-31 22:30:00    -0.681828
2022-12-31 23:00:00    -2.944016
2022-12-31 23:30:00    -3.356166
Name: 0, Length: 17520, dtype: float64

In [ ]:
xr.Dataset(
    data_vars= {
        'temperatures' : timeseries_df, 
        'avg_temp' : avg_temp
    }
)

In [26]:
GEO_WEATHER = xr.load_dataset(os.path.join(pvdeg.TEST_DATA_DIR, "summit-weather.nc"))
GEO_META = pd.read_csv(os.path.join(pvdeg.TEST_DATA_DIR, "summit-meta.csv"), index_col=0)

In [23]:
template = pvdeg.geospatial.output_template(
    ds_gids=GEO_WEATHER,
    shapes={
        'temperatures': ('gid', 'time'),
        'avg_temp' : ('gid',),
    }
)

In [49]:
def mixed_res_dataset(weather_df, meta):
    """
    geospatial test function. returns have mixed dimensions. which are correctly stored in a xr.Dataset
    """

    return xr.Dataset(
        data_vars={
            "temperatures" : (('time'), np.full((8760,), fill_value=80)),
            "avg_temp": 80
        },
        coords={
            'time' : pd.date_range(start="2001-01-01", periods=8760, freq='1h')
        }
    )

In [51]:
template = pvdeg.geospatial.output_template(
    ds_gids=GEO_WEATHER,
    shapes={
        'temperatures': ('gid', 'time'),
        'avg_temp' : ('gid',),
    }
)


pvdeg.geospatial.analysis(
    weather_ds=GEO_WEATHER,
    meta_df=GEO_META,
    func=mixed_res_dataset,
    template=template
)

<xarray.Dataset> Size: 6MB
Dimensions:       (latitude: 8, longitude: 10, time: 8760)
Coordinates:
  * latitude      (latitude) float64 64B 39.41 39.45 39.53 ... 39.69 39.81 39.89
  * longitude     (longitude) float64 80B -106.4 -106.3 -106.3 ... -105.9 -105.9
  * time          (time) datetime64[ns] 70kB 2001-01-01 ... 2001-12-31T23:00:00
Data variables:
    temperatures  (time, latitude, longitude) float64 6MB nan nan ... nan nan
    avg_temp      (latitude, longitude) float64 640B nan nan nan ... nan nan nan